In [ ]:
# API Keys


In [ ]:
import re
from typing import List, Dict, Tuple
from huggingface_hub import InferenceClient
from transformers import pipeline
from datasets import load_dataset


Device set to use cuda:0
/home/cyra/code/MindPal_Backend/AI/venv/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# from google import genai

# client = genai.Client(api_key=GOOGLE_API_KEY)

# response = client.models.generate_content(
#     model="gemini-2.5-pro", contents=prompt
# )
# print(response.text)

In [ ]:
class MindPal_Pipeline:
    def __init__(self):
        self.client = InferenceClient(
            api_key=HUGGING_FACE_TOKEN,
            provider="auto"   # Automatically selects best provider
        )
        self.emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
        self.SLANG_MAP = self.load_slang_dataset()

    def normalize_text(self, text: str) -> str:
        t = text.strip()
        # basic normalization
        t = re.sub(r"\s+", " ", t)
        return t

    # Load slang dataset into a mapping: slang -> canonical meaning(s) (examples)
    def load_slang_dataset(self) -> Dict[str, List[str]]:
        """
        Try to fetch a structured GenZ slang mapping from available datasets.
        """
        try:
            # Attempt to load MLBtrio genz-slang-dataset using datasets or Hugging Face
            ds = load_dataset("MLBtrio/genz-slang-dataset", split="train")
            slang_map = {}
            # dataset may have 'slang' and 'meaning' fields (check variety)
            for row in ds:
                key = row.get("Slang")
                desc = row.get("Description")

                if key:
                    slang_map[key.lower()] = [desc] if desc else []
            if slang_map:
                return slang_map
        except Exception as e:
            print(f"Failed to load GenZ slang dataset: {e}")
            pass

    def detect_slang_candidates(self, text: str) -> List[Tuple[str, List[str]]]:
        """
        Find slang tokens present in text and return candidate meanings.
        """
        lower = text.lower()
        found = []
        for s in self.SLANG_MAP:
            # match whole word or phrase
            if re.search(r"\b" + re.escape(s) + r"\b", lower):
                found.append((s, self.SLANG_MAP[s]))
        return found

    def map_slang_in_text(self, text: str) -> Tuple[str, List[str]]:
        """
        Returns an augmented text with slang replaced by canonical meaning (in brackets).
        """
        context = text
        candidates = self.detect_slang_candidates(text)
        slang_token, meanings = candidates[0][0], candidates[0][1]

        # replace slang with "slang (meaning)" to give classifier better signal
        context = re.sub(r"\b" + re.escape(slang_token) + r"\b", f"{slang_token} ({meanings[0]})", context, flags=re.IGNORECASE)
        return context

    def emotion_detection(self, text: str) -> str:
        emotion = self.emotion_classifier(text)
        return emotion[0]

    def generate_response(self, text: str, detected_emotion: str) -> str:
        prompt = f"""
        You are MindPal, a supportive wellbeing chatbot for 13-15 year-old australian teens. 
        Your responses should be:
        - Warm, understanding, and age-appropriate
        - Validate their feelings without being condescending
        - Use language that feels natural to teens
        - Acknowledge and reflect their feeling(s)
        - Keep replies within 1-3 sentences and sound like a natural conversation
        - Encourage them to talk more, ask follow up questions and let user express their feelings
        - Encourage real-life support systems and resources
        - When appropriate and you have enough information, gently encourage the teen to talk with a trusted adult or friend
        - Avoid shaming or lecturing
        - Use emojis to express emotions
        - Do NOT encourage any dangerous behaviour or provide inappropriate information
        - Do NOT give medical or clinical advice or replace professional help
        - Do NOT overly sugarcoat things, be natural and honest when necessary
        - Do NOT let the user give out any personal information
        - When the user ask general questions that are unrelated to your purpose, politely decline to answer and redirect the conversation back

        Current emotion(s) detected: {detected_emotion}

        Always rethink and double check your answer before responding.
        When you completely understand you can start the session.
        """

        messages = [
            {"role": "system", "content": prompt},
            {"role": "user", "content": text}
        ]

        # Chat completion
        completion = self.client.chat.completions.create(
            model="deepseek-ai/DeepSeek-R1",
            messages=messages,
            temperature=0.3,
            top_p=0.9
        )

        raw_reply = completion.choices[0].message.content
        reply = raw_reply.split("</think>")[-1].strip()

        return reply
    
    def chat(self, text: str) -> str:
        text = self.normalize_text(text)
        text = self.map_slang_in_text(text)
        detected_emotion = self.emotion_detection(text)
        reply = self.generate_response(text, detected_emotion)
        return reply


In [137]:
text = "   My parents found out I'm gay, but they don't accept it, I want to leave this place asap"
MindPal_Pipeline().chat(text)

Device set to use cuda:0
/home/cyra/code/MindPal_Backend/AI/venv/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


"I'm really sorry you're going through this – it must feel incredibly painful when the people you love don't accept you. 😔 That rejection is so tough, and feeling like you want to leave is completely understandable.  \n\nBefore you make any big decisions, is there someone safe you can talk to? Maybe a friend, school counselor, or another adult you trust? You deserve support right now. 💛"

In [ ]:
TEEN_SUPPORT_PROMPT = """

Current emotion detected: {emotion}
Conversation context: {context}

Teen's message: "{message}"

Respond as a supportive friend who truly understands what they're going through.
"""

COPING_STRATEGY_PROMPT = """
Based on the teen's emotional state ({emotion}) and situation ({context}), 
recommend 2-3 age-appropriate coping strategies from our database:

{coping_strategies_db}

Format as friendly suggestions, not instructions. Include relevant resource links.
"""

PARENT_TIPS_PROMPT = """
A parent needs guidance on how to support their teenager who is experiencing {emotion}.

Recent conversation themes: {themes}
Recommended activities teen is doing: {activities}

Provide 3-4 specific, actionable tips for the parent to:
1. Approach conversations with empathy
2. Create supportive environment
3. Know when to seek professional help
4. Respect teen's autonomy while staying involved

Keep advice practical and evidence-based.
"""

'- When appropriate and you have enough information, recommend 1-2 coping strategies, or gently encourage the teen to talk with a trusted adult or friend'